# Word2vec

In [1]:
# 부모 폴더의 경로를 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='./data/ml-10M100K/')
movielens = data_loader.load()

### Gensim

Gensim은 최신 통계 기계 학습을 사용하여 비지도 주제 모델링 , 문서 색인화 , 유사성 검색 및 기타 자연어 처리 기능을 위한 오픈 소스 라이브러리 입니다.
Gensim은 성능을 위해 Python 및 Cython 으로 구현되었습니다. 
Gensim은 데이터 스트리밍 및 증분 온라인 알고리즘을 사용하여 대규모 텍스트 컬렉션을 처리하도록 설계되었으며, 
이는 메모리 내 처리만을 대상으로 하는 대부분의 다른 기계 학습 소프트웨어 패키지와 차별화됩니다.

In [3]:
# !pip install --user gensim

In [5]:
movielens

Dataset(train=        user_id  movie_id  rating   timestamp  \
1           139       122     3.0   974302621   
2           149       122     2.5  1112342322   
3           182       122     3.0   943458784   
4           215       122     4.5  1102493547   
6           281       122     3.0   844437024   
...         ...       ...     ...         ...   
132825     1045     57949     0.5  1215617256   
132826     1045     58291     0.5  1215616991   
132827     1045     59306     3.0  1215617137   
132828     1045     60286     3.0  1215617037   
132829     1047     52952     4.0  1203399887   

                                      title                        genre  \
1                          Boomerang (1992)            [Comedy, Romance]   
2                          Boomerang (1992)            [Comedy, Romance]   
3                          Boomerang (1992)            [Comedy, Romance]   
4                          Boomerang (1992)            [Comedy, Romance]   
6                

In [6]:
import gensim
import logging

movie_content = movielens.item_content.copy()

In [7]:
movie_content

,movie_id,title,genre,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie]
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin..."
...,...,...,...,...
10676,65088,Bedtime Stories (2008),"[Adventure, Children, Comedy]",NaN
10677,65091,Manhattan Melodrama (1934),"[Crime, Drama, Romance]",NaN
10678,65126,Choke (2008),"[Comedy, Drama]","[chuck palahniuk, based on book]"
10679,65130,Revolutionary Road (2008),"[Drama, Romance]",[toplist08]


In [8]:
 movie_content['tag'].fillna("").apply(list)

0        [pixar, pixar, pixar, animation, pixar, animat...
1        [for children, game, animals, joe johnston, ro...
2        [funniest movies, comedinha de velhinhos engra...
3                                             [girl movie]
4        [steve martin, pregnancy, remake, steve martin...
                               ...                        
10676                                                   []
10677                                                   []
10678                     [chuck palahniuk, based on book]
10679                                          [toplist08]
10680                                                   []
Name: tag, Length: 10681, dtype: object

In [9]:
 movie_content['genre'].apply(list)

0        [Adventure, Animation, Children, Comedy, Fantasy]
1                           [Adventure, Children, Fantasy]
2                                        [Comedy, Romance]
3                                 [Comedy, Drama, Romance]
4                                                 [Comedy]
                               ...                        
10676                        [Adventure, Children, Comedy]
10677                              [Crime, Drama, Romance]
10678                                      [Comedy, Drama]
10679                                     [Drama, Romance]
10680                                             [Comedy]
Name: genre, Length: 10681, dtype: object

In [10]:
# tag가 부여되어 있지 않은 영화가 있지만. genre는 모든 영화에 부여되어 있다
# tag와 genre를 결합한 것을 영화의 콘텐츠 정보로 해서 비슷한 영화를 찾아서 추천한다
# tag가 없는 영화에서는 NaN으로 되어 있으므로, 빈 리스트로 초기화한다
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre']

0        [pixar, pixar, pixar, animation, pixar, animat...
1        [for children, game, animals, joe johnston, ro...
2        [funniest movies, comedinha de velhinhos engra...
3                     [girl movie, Comedy, Drama, Romance]
4        [steve martin, pregnancy, remake, steve martin...
                               ...                        
10676                        [Adventure, Children, Comedy]
10677                              [Crime, Drama, Romance]
10678      [chuck palahniuk, based on book, Comedy, Drama]
10679                          [toplist08, Drama, Romance]
10680                                             [Comedy]
Name: tag_genre, Length: 10681, dtype: object

In [11]:
movie_content['tag_genre'][0]

['pixar',
 'pixar',
 'pixar',
 'animation',
 'pixar',
 'animated',
 'fun',
 'toy',
 'toys',
 'pixar',
 'rated-g',
 'pixar',
 'pixar',
 'national film registry',
 'time travel',
 'pixar',
 'pixar',
 'funny',
 'imdb top 250',
 'animation',
 'very good',
 'ya boy',
 'pixar',
 'time travel',
 'animation',
 "erlend's dvds",
 'cgi',
 'disney',
 'family',
 'pixar',
 'toys',
 'bright',
 'daring rescues',
 'fanciful',
 'heroic mission',
 'humorous',
 'light',
 'rousing',
 'toys come to life',
 'unlikely friendships',
 'warm',
 'witty',
 'pixar',
 'the boys',
 'almost favorite',
 'é˜®ä¸€é¸£',
 'animation',
 'children',
 'disney',
 'pixar',
 'clever',
 'want to see again',
 'cartoon',
 'disney',
 '3d',
 'computer animation',
 'disney animated feature',
 'pixar animation',
 'want',
 'buddy movie',
 'animation',
 'pixar',
 'tim allen',
 'tom hanks',
 'animation',
 'disney',
 'pixar',
 'toys',
 'computer animation',
 'cartoon',
 'pixar',
 'animation',
 'computer animation',
 'pixar',
 'toys',
 "tume

In [12]:
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:set(map(str, x)))

In [13]:
movie_content['tag_genre']

0        {bright, toys come to life, very good, rated-g...
1        {kirsten dunst, Adventure, children, time, her...
2        {moldy, comedinha de velhinhos engraã§ada, Rom...
3                     {Romance, girl movie, Comedy, Drama}
4        {remake, it thought it was funny but it wasn't...
                               ...                        
10676                        {Adventure, Comedy, Children}
10677                              {Romance, Drama, Crime}
10678      {based on book, Comedy, chuck palahniuk, Drama}
10679                          {Romance, toplist08, Drama}
10680                                             {Comedy}
Name: tag_genre, Length: 10681, dtype: object

In [15]:
# 태그와 장르 데이터를 사용해서, word2vec를 학습한다
tag_genre_data = movie_content.tag_genre.tolist()

In [16]:
tag_genre_data

[{'3d',
  'Adventure',
  'Animation',
  'Children',
  'Comedy',
  'Fantasy',
  'action figure',
  'action figures',
  'adventure',
  'almost favorite',
  'animated',
  'animation',
  'avi',
  'bright',
  'buddy movie',
  'buy',
  'buzz lightyear',
  'cartoon',
  'cg animation',
  'cgi',
  'children',
  'classic',
  'clever',
  'comedy',
  'computer animation',
  'daring rescues',
  'disney',
  'disney animated feature',
  'engaging',
  "erlend's dvds",
  'family',
  'fanciful',
  'fantasy',
  'first cgi film',
  'fun',
  'funny',
  'heroic mission',
  'humorous',
  'imdb top 250',
  'john lasseter',
  'kids movie',
  'light',
  'lots of heart',
  'national film registry',
  'pixar',
  'pixar animation',
  'rated-g',
  'rousing',
  'the boys',
  'tim allen',
  'time travel',
  'tom hanks',
  'toy',
  'toys',
  'toys come to life',
  "tumey's to see again",
  "tumey's vhs",
  'unlikely friendships',
  'usa',
  'very good',
  'villian hurts toys',
  'want',
  'want to see again',
  'warm'

In [18]:
model = gensim.models.word2vec.Word2Vec(tag_genre_data, vector_size=100, window=100, sg=1, hs=0, epochs=50, min_count=5)

Word2Vec 함수는 Gensim 라이브러리에서 제공하는 기능으로, 단어를 벡터로 표현하는 Word2Vec 모델을 만듭니다.   
여기에는 아래 인자값이 사용됩니다.

- tag_genre_data : 이것은 단어를 학습하기 위한 문장들의 리스트입니다. 각 문장은 단어의 리스트로 표현되어야 합니다.

- vector_size : 이것은 단어 벡터의 크기를 결정합니다. 즉, 각 단어가 몇 차원의 벡터로 표현될지를 지정합니다. 일반적으로 100차원 또는 300차원이 많이 사용됩니다.

- window : 이것은 주변 단어의 개수를 지정합니다. 주어진 단어의 좌우로 몇 개의 단어를 볼 것인지를 의미합니다. 예를 들어, window=5는 좌우 5개의 단어를 참조합니다.

- sg : 이것은 Word2Vec 모델을 학습할 때 사용되는 알고리즘을 선택합니다. sg=1은 Skip-gram 알고리즘을 사용하고, sg=0은 CBOW (Continuous Bag of Words) 알고리즘을 사용합니다. 일반적으로 Skip-gram 알고리즘이 더 성능이 좋다고 알려져 있습니다.

- hs : 이것은 Hierarchical Softmax를 사용할지 여부를 결정합니다. hs=1은 사용하고, hs=0은 사용하지 않습니다. Hierarchical Softmax는 계산 효율을 높이기 위한 방법 중 하나입니다.

- epochs : 이것은 전체 데이터셋을 몇 번 반복해서 학습할지를 결정합니다. 에포크(epoch)는 전체 데이터셋에 대해 한 번 학습을 완료한 상태를 의미합니다.

- min_count : 이것은 모델이 학습에 사용할 단어의 최소 등장 빈도를 결정합니다. 이 값보다 빈도가 낮은 단어들은 무시됩니다. 일반적으로 너무 적게 등장한 단어는 모델의 성능에 도움이 되지 않으므로 이 값을 설정하여 제거하는 것이 좋습니다.

따라서 주어진 코드는 단어 벡터의 크기가 100이고 주변 단어를 100개까지 고려하는 Skip-gram 모델을 학습하며, 50번의 에포크 동안 데이터셋을 사용하여 학습합니다. 단어의 등장 빈도가 5보다 작은 단어는 무시됩니다. Hierarchical Softmax는 사용하지 않습니다.

In [19]:
# anime 태그와 비슷한 태그를 확인한다
model.wv.most_similar('anime')

[('zibri studio', 0.7914258241653442),
 ('studio ghibli', 0.7864949107170105),
 ('miyazaki', 0.7724219560623169),
 ('pelicula anime', 0.7545306086540222),
 ('hayao miyazaki', 0.7425793409347534),
 ('japan', 0.6656797528266907),
 ('Animation', 0.5605572462081909),
 ('curse', 0.4687414765357971),
 ('wilderness', 0.46351557970046997),
 ('samurai', 0.4605865776538849)]

In [20]:
# Word2vecContent 추천
from src.word2vec import Word2vecRecommender
recommender = Word2vecRecommender()
recommend_result = recommender.recommend(movielens)

In [21]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.010, Recall@K=0.034
